# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import pandas as pd
import re
import pickle
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import joblib

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [5]:
# load data from database
# engine = create_engine('sqlite:///{}'.format(database_filepath))
engine = create_engine('sqlite:///DisasterResponse_new.db')
df = pd.read_sql_table('table',engine)  
X = df['message']
Y = df.drop(columns=['id','genre','message','original'],axis=1)

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    # normalize
    # text = text[0]
    # print(text)
    text = re.sub(r"[^a-zA-Z0-9]"," ",text)
    # tokenize
    words = word_tokenize(text)
    # stop word removal
    words = [w for w in words if w not in stopwords.words("english")]
    # stem & lemmatized
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in words:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    
    return clean_tokens

In [7]:
# Test function
text = 'Weather update - a cold front from Cuba that'
tokenize(text)

['weather', 'update', 'cold', 'front', 'cuba']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(AdaBoostClassifier())),
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train,X_test,y_train,y_test = train_test_split(X,Y)

pipeline.fit(X_train,y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000022B53CB8E50>)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
predicted = pipeline.predict(X_test)

In [11]:
for i, col in enumerate(y_test):
    print(col)
    print(classification_report(y_test[col],predicted[:, i]))

related
              precision    recall  f1-score   support

           0       0.65      0.20      0.30      1519
           1       0.79      0.97      0.87      4974
           2       0.25      0.06      0.09        52

    accuracy                           0.78      6545
   macro avg       0.57      0.41      0.42      6545
weighted avg       0.76      0.78      0.73      6545

request
              precision    recall  f1-score   support

           0       0.90      0.97      0.94      5447
           1       0.76      0.50      0.60      1098

    accuracy                           0.89      6545
   macro avg       0.83      0.73      0.77      6545
weighted avg       0.88      0.89      0.88      6545

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6515
           1       0.00      0.00      0.00        30

    accuracy                           0.99      6545
   macro avg       0.50      0.50      0.50      654

### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x0000022B53CB8E50>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x0000022B53CB8E50>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=AdaBoostClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf_

In [13]:
parameters = {'tfidf__use_idf': (True, False), 
              'clf__estimator__n_estimators': [20,80],
              'clf__estimator__base_estimator': [SVC(), DecisionTreeClassifier()]
              }

cv = GridSearchCV(pipeline, param_grid=parameters)

cv.fit(X_train, y_train)
predicted = cv.predict(X_test)

C:\Users\a0921\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
20 fits failed out of a total of 40.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\a0921\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\a0921\anaconda3\lib\site-packages\sklearn\pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\a0921\anaconda3\lib\site-packages\sklearn\multioutput.py", line 434, in fit
    super().fit(X, Y, sample_weight, **fit_params)
  File "C:\User

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
for i, col in enumerate(y_test):
    print(col)
    print(classification_report(y_test[col],predicted[:, i]))

related
              precision    recall  f1-score   support

           0       0.59      0.49      0.54      1519
           1       0.85      0.89      0.87      4974
           2       0.24      0.42      0.31        52

    accuracy                           0.79      6545
   macro avg       0.56      0.60      0.57      6545
weighted avg       0.79      0.79      0.79      6545

request
              precision    recall  f1-score   support

           0       0.89      0.97      0.93      5447
           1       0.75      0.42      0.53      1098

    accuracy                           0.88      6545
   macro avg       0.82      0.69      0.73      6545
weighted avg       0.87      0.88      0.86      6545

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6515
           1       0.00      0.00      0.00        30

    accuracy                           0.99      6545
   macro avg       0.50      0.50      0.50      654

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [15]:
with open('DisasterResponse_model.pkl', 'wb') as f:
    pickle.dump(cv, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.